# NLP_언어 판정하기
> 이하 내용은 [파이썬을 이용한 머신러닝, 딥러닝 실전 앱 개발](https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=187073338)의 4장 '자연어 처리하기'를 기반으로 한다.

- 머신러닝을 활용하여 한국어, 영어, 일본어를 판정하는 모델을 만들어 본다.

## 사용하는 문자가 다른 언어 판정하기

In [1]:
# 라이브러리 불러오기
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [6]:
# Unicode 코드 포인트로 출현 빈도 판정
def count_codePoint(str):
    # Unicode 코드 포인트를 저장할 배열 준비
    counter = np.zeros(65535)
    
    for i in range(len(str)):
        # 각 문자를 Unicode 코드 포인트로 변환하기
        code_point = ord(str[i])
        if code_point > 65535:
            continue
            
        # 출연 횟수 세기
        counter[code_point] += 1
        
    # 각 요소를 문자 수로 나눠 정규화하기
    counter = counter / len(str)
    return counter

# 학습 전용 데이터 준비
ko_str = '안녕하세요. 만나서 반갑습니다.'
ja_str = 'はじめまして。どうぞよろしくお願いいたします。'
en_str = 'Hello. Nice to see you.'
cn_str = '您好，认识你很高兴。'

x_train = [count_codePoint(ko_str), count_codePoint(ja_str), count_codePoint(en_str), count_codePoint(cn_str)]
y_train = ['ko', 'ja', 'en', 'cn']


# 학습하기
clf = GaussianNB()
clf.fit(x_train, y_train)


# 평가 전용 데이터 준비하기
ko_test_str = '장난하냐?'
ja_test_str = 'ふざけているのか？'
en_test_str = 'Are you fucking kidding me?'
cn_test_str = '你闹我吗？'

x_test = [count_codePoint(ko_test_str), count_codePoint(ja_test_str), count_codePoint(en_test_str), count_codePoint(cn_test_str)]
y_test = ['ko', 'ja', 'en', 'cn']


# 평가하기
y_pred = clf.predict(x_test)
print(y_pred)
print('정답률 = ', accuracy_score(y_test, y_pred))


['ko' 'ja' 'en' 'cn']
정답률 =  1.0


## 사용하는 문자가 같은 언어 판정하기

In [7]:
# 라이브러리 추가로 불러오기
import glob

In [12]:
# Unicode의 코드 포인트로 출현 빈도 판정하기
def count_codePoint(str):
    # Unicode 코드 포인트를 저장할 배열 준비하기
    counter = np.zeros(65535)
    
    for i in range(len(str)):
        # 각 문자를 Unicode 코드 포인트로 변환하기
        code_point = ord(str[i])
        if code_point > 65535:
            continue
        # 출연 횟수 세기
        counter[code_point] += 1
        
    # 각 요소를 문자 수로 나눠 정규화하기
    counter = counter / len(str)
    return counter


# 학습 전용 데이터 준비하기
index = 0
x_train = []
y_train = []
for file in glob.glob('./train/*.txt'):
    # 언어 정보를 추출하고 레이블로 지정하기
    y_train.append(file[8:10])
    
    # 파일 내부의 문자열을 모두 추출한 뒤 빈도 배열로 변환한 뒤 입력 데이터로 사용
    file_str = ''
    for line in open(file, 'r', encoding='utf-8'): # encoding을 추가해주지 않을 경우 unicodedecodeerror가 발생한다!
        file_str = file_str + line
    x_train.append(count_codePoint(file_str))
    
# 학습하기
clf = GaussianNB()
clf.fit(x_train, y_train)


# 평가 데이터 준비하기
index = 0
x_test = []
y_test = []
for file in glob.glob('./test/*.txt'):
    # 언어 정보를 추출하고 레이블로 지정하기
    y_test.append(file[7:9])
    
    # 파일 내부의 문자열을 모두 추출한 후 빈도 배열로 변환한 뒤 입력 데이터로 사용
    file_str = ''
    for line in open(file, 'r', encoding='utf-8'):
        file_str = file_str + line
    x_test.append(count_codePoint(file_str))
    
# 평가하기
y_pred = clf.predict(x_test)
print(y_pred)
print('정답률 = ', accuracy_score(y_test, y_pred))

['de' 'en' 'es']
정답률 =  1.0
